In [3]:
from sklearn.datasets import make_blobs
from keras.src.saving import load_model
import pickle

In [10]:
scalar_path = "../models/scalar.pickle"
path_model = "../models/model.keras"
# load model
model = load_model(path_model)
# open scalar
with open(scalar_path, "rb") as f:
    scalar = pickle.load(f)
# new instances where we do not know the answer
Xnew, _ = make_blobs(n_samples=3, centers=2, n_features=2, random_state=1)
Xnew = scalar.transform(Xnew)
# make a prediction
# ynew = model.predict_step(Xnew)
ynew = model.predict(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
	print("X=%s, Predicted=%s" % (Xnew[i], ynew[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
X=[0.89337759 0.65864154], Predicted=[0.17742321]
X=[0.29097707 0.12978982], Predicted=[0.9963976]
X=[0.78082614 0.75391697], Predicted=[0.17742321]
